In [ ]:


# ==========================================================
# MAGI vs LASSO with Top (ceil(n_patients / 10)) Features
# - Loads MAGI coefficients
# - Samples up to 1,000 positives + 4× negatives (or all pos if <1000, neg=4×pos)
# - Selects top-N features by prevalence in the SAMPLE (N = ceil(n_rows/10)),
#   prioritizing MAGI-coefficient features to ensure overlap when possible
# - Scores MAGI (intercept-only fallback) and fits LASSO (5-fold CV)
# - Saves single ROCs and a combined ROC overlay (MAGI vs LASSO)
# - Always writes a summary CSV row per target
# ==========================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import load_npz
from scipy.special import expit
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import (
    roc_curve, roc_auc_score, average_precision_score,
    precision_recall_curve, auc as auc_area
)


In [ ]:

# ---------- UTILS ----------
def banner(txt):
    bar = "=" * max(12, len(txt) + 4)
    print(f"\n{bar}\n{txt}\n{bar}")

def subhead(txt):
    print(f"\n--- {txt} ---")

def safe_name(s: str) -> str:
    return "".join(ch if ch.isalnum() or ch in ("-", "_") else "_" for ch in s)

def qtiles(x):
    x = np.asarray(x)
    return np.quantile(x, [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1.0])

def preview_active_codes(X_csr, feature_codes, row_indices, k=8):
    for i in row_indices:
        start, end = X_csr.indptr[i], X_csr.indptr[i+1]
        cols = X_csr.indices[start:end]
        codes_list = [feature_codes[j] for j in cols[:k]]
        print(f"   row {i}: n_active={len(cols)}  sample_active={codes_list}")

def warn_if_exceeds_cap(n_used: int, n_rows: int, label: str = "features"):
    cap = int(np.ceil(n_rows / 10.0))
    if n_used > cap:
        print(f"[WARN] {label}: {n_used} > 1/10 of cases ({cap}). "
              f"Consider tightening selection or thresholds.")
    return cap

def load_magi_betas(coef_csv):
    df = pd.read_csv(coef_csv)
    def pick(df, opts):
        for c in opts:
            if c in df.columns:
                return c
        raise KeyError(f"Missing any of {opts} in {coef_csv}. Found: {list(df.columns)}")
    code_col = pick(df, ["concept_code","standard_concept_code","predictor","feature","term","name"])
    beta_col = pick(df, ["coef","coefficient","beta","estimate","b","value"])
    df[code_col] = df[code_col].astype(str).str.strip()
    is_int = df[code_col].str.lower().isin(["(intercept)","intercept","const","(const)","bias"])
    intercept = float(df.loc[is_int, beta_col].iloc[0]) if is_int.any() else 0.0
    coef_map  = dict(zip(df.loc[~is_int, code_col], df.loc[~is_int, beta_col].astype(float)))
    return intercept, coef_map

def sample_fixed_pos_neg(y, n_pos=1000, seed=42):
    """
    If <1000 positives, take ALL positives, then take 4× as many negatives
    (capped by availability). Same as your intent.
    """
    rng = np.random.default_rng(seed)
    pos_idx = np.where(y == 1)[0]
    neg_idx = np.where(y == 0)[0]
    if pos_idx.size == 0:
        raise ValueError("No positives for this target.")

    take_pos = min(n_pos, pos_idx.size)
    take_neg = min(4 * take_pos, neg_idx.size)

    sel_pos = rng.choice(pos_idx, size=take_pos, replace=False)
    sel_neg = rng.choice(neg_idx, size=take_neg, replace=False)
    sel = np.concatenate([sel_pos, sel_neg])
    rng.shuffle(sel)
    return sel

def drop_constants_and_duplicates_for_sample(X_csr, feature_codes):
    n_rows, n_cols = X_csr.shape
    nnz = np.asarray((X_csr != 0).sum(axis=0)).ravel()
    keep_mask = (nnz > 0)

    X_csc = X_csr[:, keep_mask].tocsc()
    sub_keep = np.ones(X_csc.shape[1], dtype=bool)
    seen = {}
    for j in range(X_csc.shape[1]):
        s, e = X_csc.indptr[j], X_csc.indptr[j+1]
        idx = X_csc.indices[s:e]
        dat = X_csc.data[s:e]
        key = (idx.tobytes(), dat.tobytes())
        if key in seen:
            sub_keep[j] = False
        else:
            seen[key] = j

    keep_idx = np.where(keep_mask)[0]
    keep_mask_final = np.zeros(n_cols, dtype=bool)
    keep_mask_final[keep_idx[sub_keep]] = True

    X_red = X_csr[:, keep_mask_final]
    feats_red = feature_codes[keep_mask_final]
    dropped = n_cols - int(keep_mask_final.sum())
    print(f"[INFO] LASSO collinearity cleanup: kept {X_red.shape[1]:,}/{n_cols:,} features "
          f"(dropped {dropped:,} all-zero/duplicate).")
    return X_red, feats_red, keep_mask_final

def select_features_current_policy(
    X_train, y_train, feature_codes, coef_map,
    n_keep, rank="beta", require_present=True,
    min_total_count=1, min_pos_carriers=0, verbose=False
):
    """
    Matches your current C2 path:
      - MAGI-only (codes present in coef_map), NO FILL
      - Optionally require presence on TRAIN split
      - Rank by |beta| (default) or by TRAIN prevalence
      - Keep up to n_keep (ceil(n_train/10) upstream), fewer if not enough

    Returns keep_idx (absolute col indices w.r.t. X_train/feature_codes).
    """
    # MAGI mapping across ALL predictor columns
    mapped_mask = np.array([c in coef_map for c in feature_codes], dtype=bool)
    cand_idx = np.where(mapped_mask)[0]
    if cand_idx.size == 0:
        if verbose: print("[SEL] No MAGI-mapped features available.")
        return np.array([], dtype=int)

    # Presence/support checks on TRAIN
    if require_present or (min_total_count > 1) or (min_pos_carriers > 0):
        nnz_all = np.asarray((X_train != 0).sum(axis=0)).ravel()  # per-column
        keep = mapped_mask.copy()
        if require_present:
            keep &= (nnz_all >= max(1, min_total_count))
        if min_pos_carriers > 0 and y_train is not None and y_train.any():
            pos_rows = np.where(y_train == 1)[0]
            nnz_pos = np.asarray((X_train[pos_rows, :] != 0).sum(axis=0)).ravel()
            keep &= (nnz_pos >= min_pos_carriers)
        cand_idx = np.where(keep)[0]
        if cand_idx.size == 0:
            if verbose: print("[SEL] After presence/support filters, no features remain.")
            return np.array([], dtype=int)

    # Ranking
    if rank == "prevalence":
        nnz = np.asarray((X_train != 0).sum(axis=0)).ravel()
        order = cand_idx[np.argsort(-nnz[cand_idx])]  # DESC by TRAIN prevalence
    elif rank == "beta":
        betas = np.array([coef_map[feature_codes[i]] for i in cand_idx], dtype=float)
        order = cand_idx[np.argsort(-np.abs(betas))]  # DESC by |β|
    else:
        raise ValueError("rank must be 'beta' or 'prevalence'")

    keep_idx = order[:min(n_keep, order.size)]
    if verbose:
        print(f"[SEL] MAGI cand={order.size:,}  keep_idx={keep_idx.size:,}  n_keep={n_keep}")
    return keep_idx

# --- NEW: build top-N by prevalence, prioritizing MAGI features to ensure overlap when possible
def build_topN_with_magi_overlap(X_sub, feature_codes, coef_map, n_keep):
    """
    1) Rank all sample features by prevalence (non-zero counts).
    2) Take as many MAGI-supported features (present in sample) as possible first.
    3) If fewer than n_keep, fill with highest-prevalence non-MAGI features.
    Returns X_top, feats_top, chosen_idx, n_magi_overlap
    """
    nnz_all = np.asarray((X_sub != 0).sum(axis=0)).ravel()
    order_all = np.argsort(-nnz_all)  # descending by prevalence

    coef_keys = np.array(list(coef_map.keys()), dtype=str)
    is_coef = np.isin(feature_codes, coef_keys)

    # MAGI-supported features present in sample, ordered by prevalence
    coef_order = order_all[is_coef[order_all]]
    chosen = list(coef_order[:n_keep])

    # fill remainder with top non-MAGI features
    if len(chosen) < n_keep:
        noncoef_order = order_all[~is_coef[order_all]]
        fill_needed = n_keep - len(chosen)
        chosen.extend(list(noncoef_order[:fill_needed]))

    chosen = np.array(chosen, dtype=int)
    X_top = X_sub[:, chosen]
    feats_top = feature_codes[chosen]
    n_magi_overlap = int(np.isin(feats_top, coef_keys).sum())
    return X_top, feats_top, chosen, n_magi_overlap

# --- Single-model ROC (bold axes; no title)
def plot_roc(y_true, p_hat, title, out_png, out_svg):
    fpr, tpr, _ = roc_curve(y_true, p_hat)
    auc_val = roc_auc_score(y_true, p_hat)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC={auc_val:.3f}")
    plt.plot([0,1], [0,1], linestyle="--", linewidth=1)
    plt.xlabel("False Positive Rate", fontsize=16, fontweight="bold")
    plt.ylabel("True Positive Rate",  fontsize=16, fontweight="bold")
    plt.xticks(fontsize=14, fontweight="bold")
    plt.yticks(fontsize=14, fontweight="bold")
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(1.4)
    plt.legend(loc="lower right", fontsize=12)
    plt.tight_layout()
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.savefig(out_svg, bbox_inches="tight")
    plt.close()
    return float(auc_val)

# --- NEW: Dual ROC (MAGI vs LASSO)
def plot_roc_dual(y_true, p1, p2, label1, label2, out_png, out_svg):
    fpr1, tpr1, _ = roc_curve(y_true, p1)
    fpr2, tpr2, _ = roc_curve(y_true, p2)
    auc1 = roc_auc_score(y_true, p1)
    auc2 = roc_auc_score(y_true, p2)
    plt.figure()
    plt.plot(fpr1, tpr1, label=f"{label1} (AUC={auc1:.3f})")
    plt.plot(fpr2, tpr2, label=f"{label2} (AUC={auc2:.3f})")
    plt.plot([0,1], [0,1], linestyle="--", linewidth=1)
    plt.xlabel("False Positive Rate", fontsize=16, fontweight="bold")
    plt.ylabel("True Positive Rate",  fontsize=16, fontweight="bold")
    plt.xticks(fontsize=14, fontweight="bold")
    plt.yticks(fontsize=14, fontweight="bold")
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(1.4)
    plt.legend(loc="lower right", fontsize=12)
    plt.tight_layout()
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.savefig(out_svg, bbox_inches="tight")
    plt.close()
    return float(auc1), float(auc2)

# ---------- LOAD DESIGN ----------
banner("LOAD DESIGN")
X_full  = load_npz(f"{BASE}/Lasso_X.npz").tocsr().astype(np.float32)
persons = pd.read_csv(f"{BASE}/person_index.csv")["person_id"].astype(str).to_numpy()
codes   = pd.read_csv(f"{BASE}/code_index.csv")["concept_code"].astype(str).to_numpy()
print(f"[INFO] Matrix: persons={X_full.shape[0]:,}  codes={X_full.shape[1]:,}")
if len(persons) != X_full.shape[0] or len(codes) != X_full.shape[1]:
    raise ValueError("[ERROR] person/code indices do not match matrix shape.")

# ---------- RUN PER TARGET ----------
summary = []
for tcode in TARGETS:
    pretty = TARGET_NAME.get(tcode, tcode)
    safe   = safe_name(pretty)  # define early so all sections can use it
    banner(f"TARGET {tcode} — {pretty}")

    # A) Labels
    subhead("A) Label vector from full design")
    idx_y = np.where(codes == tcode)[0]
    if idx_y.size == 0:
        print(f"[SKIP] Target not found in code_index.csv → {tcode}")
        continue
    y_full = X_full[:, idx_y[0]].toarray().ravel().astype(np.int8)
    print(f"[INFO] y_full: n={y_full.size:,}  pos={int(y_full.sum()):,}  prev={y_full.mean():.4f}")

    # B) Predictors
    subhead("B) Predictor matrix (keep all columns except DV)")
    mask_pred = (codes != tcode)
    X = X_full[:, mask_pred]
    feature_codes = codes[mask_pred]
    print(f"[INFO] Predictors: persons={X.shape[0]:,}  features={X.shape[1]:,}")
    print(f"[CHECK] DV in features? {tcode in feature_codes} (should be False)")

    # C) Sampling rule: up to 1,000 pos; neg = 4×pos
    subhead("C) Sampling (1,000 positives + 4× negatives)")
    sel = sample_fixed_pos_neg(y_full, n_pos=1000, seed=RNG_SEED)
    X_sub       = X[sel, :]
    y_sub       = y_full[sel].astype(np.int8)
    persons_sub = persons[sel]
    n_rows      = X_sub.shape[0]
    n_pos_sub   = int(y_sub.sum()); n_neg_sub = n_rows - n_pos_sub
    print(f"[INFO] subset: n={n_rows:,}  pos={n_pos_sub:,}  neg={n_neg_sub:,}  "
          f"ratio≈{(n_neg_sub/max(n_pos_sub,1)):.2f}:1  PR-baseline={y_sub.mean():.4f}")
    try:
        preview_active_codes(X_sub, feature_codes, row_indices=range(min(3, n_rows)), k=8)
    except Exception as e:
        print(f"[WARN] preview_active_codes failed: {e}")

    # D) Load MAGI coefficients (MUST be before C2; used to prioritize)
    subhead("D) Load MAGI coefficients")
    coef_csv = COEF_PATTERN.format(target=tcode)
    if not os.path.exists(coef_csv):
        print(f"[SKIP] Coef file missing: {coef_csv}")
        continue
    intercept, coef_map = load_magi_betas(coef_csv)
    print(f"[INFO] Coefs: intercept={intercept:.6f}  n_features={len(coef_map):,}")
    for k,(cc,bb) in enumerate(list(coef_map.items())[:5]):
        print(f"   beta[{cc}] = {bb:.6f}")

    # C2) Pick N = ceil(n/10) by prevalence, prioritizing MAGI features
    subhead("C2) Keep top ceil(n_patients/10) features (prioritize MAGI features)")
    N_keep = int(np.ceil(n_rows / 10.0))
    X_top, feats_top, kept_idx, magi_in_top = build_topN_with_magi_overlap(
        X_sub, feature_codes, coef_map, N_keep
    )
    print(f"[INFO] Top-N features: kept {X_top.shape[1]} / {feature_codes.size} (N={N_keep}); "
          f"MAGI features within top-N = {magi_in_top}")

    # C3) Cleanup once (drop all-zero/duplicates) — shared by MAGI & LASSO
    X_used, feats_used, _ = drop_constants_and_duplicates_for_sample(
        X_top.tocsr().astype(np.float32), feats_top
    )
    n_used = X_used.shape[1]
    magi_keys = set(coef_map.keys())
    n_magi_used = int(sum(f in magi_keys for f in feats_used))
    print(f"[INFO] Features after shared cleanup: {n_used} (of which MAGI={n_magi_used})")
    
    cap_1of10 = warn_if_exceeds_cap(n_used, n_rows, label="Features after shared cleanup")

    # After C3:
    magi_mask = np.array([f in coef_map for f in feats_used], dtype=bool)
    X_used = X_used[:, magi_mask]
    feats_used = feats_used[magi_mask]
    print(f"[INFO] Enforcing: {X_used.shape[1]} features remain")

    # E) Align & score (MAGI) — SAME cleaned features as LASSO
    betas_vec = np.array([coef_map.get(f, 0.0) for f in feats_used], dtype=float)  # 0.0 for non-MAGI cols
    lp = intercept + X_used.dot(betas_vec)
    p_hat_magi = expit(np.asarray(lp).ravel())
    auc_magi = roc_auc_score(y_sub, p_hat_magi)
    print(f"[RESULT] MAGI AUC={auc_magi:.4f} (baseline={y_sub.mean():.4f})")

    # E2) Save MAGI non-zero coefficients actually used (post-cleanup)
    magi_nz_mask  = betas_vec != 0
    magi_nz_feats = np.array(feats_used)[magi_nz_mask]
    magi_nz_betas = betas_vec[magi_nz_mask]
    magi_coef_used_df = (
        pd.DataFrame({"feature": magi_nz_feats, "beta": magi_nz_betas})
          .assign(abs_beta=lambda df: df["beta"].abs())
          .sort_values("abs_beta", ascending=False)
          .drop(columns="abs_beta")
    )
    magi_coef_used_df.loc[-1] = {"feature": "(intercept)", "beta": float(intercept)}
    magi_coef_used_df.index = magi_coef_used_df.index + 1
    magi_used_csv = os.path.join(OUT_DIR, f"magi_coef_used_{safe}.csv")
    magi_coef_used_df.to_csv(magi_used_csv, index=False)
    print(f"[SAVE] MAGI used coefficients → {magi_used_csv} "
          f"(nonzero={magi_nz_feats.size} of {len(feats_used)})")

    # F) Distribution (optional)
    subhead("F) Metrics & probability distribution (MAGI)")
    q = qtiles(p_hat_magi)
    print(f"[RESULT] MAGI AUC={auc_magi:.4f}  (baseline={y_sub.mean():.4f})")
    print(f"[DIST] prob quantiles: min={q[0]:.4g}, p1={q[1]:.4g}, p25={q[2]:.4g}, "
          f"median={q[3]:.4g}, p75={q[4]:.4g}, p99={q[5]:.4g}, max={q[6]:.4g}")

    # G) Save MAGI predictions
    subhead("G) Save per-person predictions (MAGI)")
    pred_csv = os.path.join(CSV_DIR, f"pred_{safe}.csv")
    pd.DataFrame({"person_id": persons_sub, "y_true": y_sub.astype(int), "prob_magi": p_hat_magi}).to_csv(pred_csv, index=False)
    print(f"[SAVE] predictions → {pred_csv}")

    # H) Single ROC (MAGI)
    subhead("H) ROC plots (MAGI) PNG/SVG")
    png_path = os.path.abspath(os.path.join(PNG_DIR, f"ROC_{safe}.png"))
    svg_path = os.path.abspath(os.path.join(PNG_DIR, f"ROC_{safe}.svg"))
    _ = plot_roc(y_sub, p_hat_magi, pretty, png_path, svg_path)
    print(f"[SAVE] ROC → {png_path}")
    print(f"[SAVE] ROC → {svg_path}")

    # I) LASSO: 5-fold CV on the SAME cleaned features (NO extra cleanup)
    subhead("I) LASSO: 5-fold CV on SAME cleaned features")
    try:
        if X_used.shape[1] == 0:
            print("[SKIP] No usable features; LASSO fallback to constant.")
            lasso_cv_auc = np.nan; lasso_auc = np.nan
            lasso_pred_csv = ""; lasso_coef_csv = ""
            p_hat_lasso = np.full(n_rows, float(y_sub.mean()), dtype=float)
        else:
            clf = LogisticRegressionCV(
                Cs=np.logspace(-3, 3, 12),
                cv=5,
                penalty="l1",
                solver="saga",
                scoring="roc_auc",
                max_iter=2000,
                n_jobs=-1,
                random_state=RNG_SEED,
                refit=True,
                fit_intercept=True,
            ).fit(X_used, y_sub)

            scores_mat = clf.scores_[1]
            mean_auc_per_C = scores_mat.mean(axis=0)
            best_idx = int(np.argmax(mean_auc_per_C))
            lasso_cv_auc = float(mean_auc_per_C[best_idx])
            best_C = float(np.atleast_1d(clf.C_)[0])
            print(f"[MODEL] LASSO best_C={best_C:.6g}  CV-AUC={lasso_cv_auc:.4f}")

            p_hat_lasso = clf.predict_proba(X_used)[:, 1]
            lasso_auc   = roc_auc_score(y_sub, p_hat_lasso)
            print(f"[RESULT] LASSO Refit AUC={lasso_auc:.4f} (baseline={y_sub.mean():.4f})")

            # Save LASSO predictions
            lasso_pred_csv = os.path.join(CSV_DIR, f"pred_{safe}_LASSO.csv")
            pd.DataFrame({"person_id": persons_sub, "y_true": y_sub.astype(int), "prob_lasso": p_hat_lasso}).to_csv(lasso_pred_csv, index=False)
            print(f"[SAVE] LASSO predictions → {lasso_pred_csv}")

            # Save non-zero LASSO coefficients (aligned to feats_used)
            coef = clf.coef_.ravel(); intercept_l = float(clf.intercept_.ravel()[0])
            nz = np.where(coef != 0)[0]
            coef_df = (pd.DataFrame({"feature": np.array(feats_used)[nz], "coef": coef[nz]})
                       .sort_values("coef", key=np.abs, ascending=False))
            coef_df.loc[-1] = {"feature": "(intercept)", "coef": intercept_l}
            coef_df.index = coef_df.index + 1
            lasso_coef_csv = os.path.join(OUT_DIR, f"lasso_coef_{safe}.csv")
            coef_df.to_csv(lasso_coef_csv, index=False)
            print(f"[SAVE] LASSO coefficients → {lasso_coef_csv} (nonzero={len(nz)} of {len(coef)})")

            # ROC (LASSO)
            lasso_roc_png = os.path.abspath(os.path.join(PNG_DIR, f"ROC_{safe}_LASSO.png"))
            lasso_roc_svg = os.path.abspath(os.path.join(PNG_DIR, f"ROC_{safe}_LASSO.svg"))
            _ = plot_roc(y_sub, p_hat_lasso, f"{pretty} (LASSO, CV=5)", lasso_roc_png, lasso_roc_svg)
            print(f"[SAVE] LASSO ROC → {lasso_roc_png}")
    except Exception as e:
        print(f"[SKIP] LASSO failed: {e}")
        lasso_cv_auc = np.nan; lasso_auc = np.nan
        lasso_pred_csv = ""; lasso_coef_csv = ""
        p_hat_lasso = np.full(n_rows, float(y_sub.mean()), dtype=float)

    # J) Combined ROC overlay (MAGI vs LASSO)
    subhead("J) Combined ROC: MAGI vs LASSO (overlay)")
    dual_png = os.path.abspath(os.path.join(PNG_DIR, f"ROC_{safe}_MAGI_vs_LASSO.png"))
    dual_svg = os.path.abspath(os.path.join(PNG_DIR, f"ROC_{safe}_MAGI_vs_LASSO.svg"))
    _ = plot_roc_dual(y_true=y_sub, p1=p_hat_magi, p2=p_hat_lasso,
                      label1="MAGI", label2="LASSO",
                      out_png=dual_png, out_svg=dual_svg)
    print(f"[SAVE] Dual ROC → {dual_png}")

    # Summary row
    summary.append({
        "target_code": tcode,
        "target_name": pretty,
        "n_cases": int(n_rows),
        "n_pos": int(n_pos_sub),
        "n_neg": int(n_neg_sub),
        "features_used": int(n_used),
        "features_used_MAGI": int(n_magi_used),
        "AUC_MAGI": float(auc_magi),
        "PR_baseline": float(y_sub.mean()),
        "coef_csv": coef_csv,
        "pred_csv_MAGI": pred_csv,
        "roc_png_MAGI": os.path.abspath(png_path),
        "LASSO_CV_AUC": float(lasso_cv_auc) if not np.isnan(lasso_cv_auc) else np.nan,
        "LASSO_Refit_AUC": float(lasso_auc) if not np.isnan(lasso_auc) else np.nan,
        "LASSO_pred_csv": lasso_pred_csv,
        "LASSO_coef_csv": lasso_coef_csv,
        "Dual_ROC_png": dual_png,
        "magi_nonzero_used": int(magi_nz_feats.size),
        "magi_used_coef_csv": magi_used_csv,
    })

